<a href="https://colab.research.google.com/github/snowgecko/datathon_team_04/blob/main/filterdata_VACs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
## Imports all the packages.
import numpy as np
import pandas as pd
import os
import pdb
import array

from IPython.core.display import display, HTML
import scipy.stats as stats
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as dates

!pip install scikit-learn
from sklearn import datasets, linear_model
import matplotlib
from sklearn.linear_model import LinearRegression

from google.colab import auth
from google.cloud import bigquery
from google.colab import files

%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# get the amsterdamumcdb package from PyPI repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb


###################SET PROJECT DEFAULTS##################
PROJECT_ID = "dynamic-reef-423509-d1" #@param {type:"string"}

# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'version1_5_0' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

# all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = "dynamic-reef-423509-d1"

auth.authenticate_user()
print('Authenticated')

config_gbq = {'query':
          {'defaultDataset': {
              "datasetId": DATASET_ID,
              "projectId": DATASET_PROJECT_ID
              },
           'Location': LOCATION}
           }

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 30000

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset="amsterdamumcdb" + "." + "version1_5_0")
# sets default client settings by re-using the previously defined config
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)



The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
Authenticated


In [9]:
from google.colab import drive
drive.mount('/content/drive')
#need to select all otherwise it errors out.

# path to base folder (right click on the directory: Datathon, and select 'Add to my Drive' in order to appear on My Drive)
drive_folder = '/content/drive/My Drive/Datathon/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Shared functions
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=DATASET_PROJECT_ID,
      dialect='standard')


In [28]:
###########################original imported file was ######################################
#3025408 -- o2 conc
#3020716, --fio2
#rslt_df_3d_fi02_2d_onwards = pd.DataFrame()
#rslt_df_3d = pd.read_csv(drive_folder + "rslt_df_3d.csv") # read the dataset to df_data

#############################################################################################
## 1] filtered to those visit_occurrence_ids with fi02 or O2 conc.
## 2] then that are in ICU for 4 days
## 3] then getting the data for 2 calendar days after the Min_Date (something wierd going on with vo_id=0)

rslt_df_3d_fi02_2d_onwards = pd.read_csv(drive_folder + "rslt_df_3d_fi02_2d_onwards.csv") # read the dataset to df_data

rslt_df_3d_fi02_idata = rslt_df_3d_fi02_2d_onwards[rslt_df_3d_fi02_2d_onwards['visit_occurrence_id'] == 344]

rslt_df_3d_fi02_idata.to_csv(drive_folder + "rslt_df_3d_fi02_idata.csv", sep=',', index=False, encoding='utf-8')


In [87]:
rslt_df_3d_fi02_idata = pd.read_csv(drive_folder + "rslt_df_3d_fi02_idata.csv") # read the dataset to df_data

##changing the following 4 fields to datetime format
rslt_df_3d_fi02_idata['measurement_datetime'] = pd.to_datetime(rslt_df_3d_fi02_idata['measurement_datetime'])
rslt_df_3d_fi02_idata['measurement_date'] = pd.to_datetime(rslt_df_3d_fi02_idata['measurement_date'])
rslt_df_3d_fi02_idata['date_min'] = pd.to_datetime(rslt_df_3d_fi02_idata['date_min'])
rslt_df_3d_fi02_idata['date_max'] = pd.to_datetime(rslt_df_3d_fi02_idata['date_max'])

##############check
rslt_df_3d_fi02_idata.dtypes



measurement_id                                 int64
person_id                                      int64
measurement_concept_id                         int64
measurement_date                      datetime64[ns]
measurement_datetime             datetime64[ns, UTC]
measurement_time                              object
measurement_type_concept_id                    int64
operator_concept_id                          float64
value_as_number                              float64
value_as_concept_id                          float64
unit_concept_id                              float64
range_low                                    float64
range_high                                   float64
provider_id                                    int64
visit_occurrence_id                            int64
visit_detail_id                              float64
measurement_source_value                      object
measurement_source_concept_id                float64
unit_source_value                             

In [65]:
#rslt_df_3d_fi02_idata = rslt_df_3d_fi02_idata.set_index("measurement_datetime")

In [88]:
###individual data (idata) for specific id
#rslt_df_3d_fi02_idata = rslt_df_3d_fi02_2d_onwards[rslt_df_3d_fi02_2d_onwards['visit_occurrence_id'] == 344]

##create new columns of int type
rslt_df_3d_fi02_idata['delta_min_fi02'] = pd.Series(dtype='int')
#rslt_df_3d_fi02_idata['delta_prev_fi02'] = pd.Series(dtype='int')
rslt_df_3d_fi02_idata['delta_unst_dt'] = pd.Series(dtype='datetime64[ns]')
rslt_df_3d_fi02_idata['delta_unst_days'] = pd.Series(dtype='int')
#datetime64[ns]

rslt_df_3d_fi02_idata.head(50)

,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_time,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,...,unit_source_concept_id,value_source_value,measurement_event_id,meas_event_field_concept_id,date_min,date_max,diff,delta_min_fi02,delta_unst_dt,delta_unst_days
0,85171342,303,3025408,2006-01-02,2006-01-02 00:54:00+00:00,00:54:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
1,85171383,303,3025408,2006-01-02,2006-01-02 01:52:00+00:00,01:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
2,85171421,303,3025408,2006-01-02,2006-01-02 02:52:00+00:00,02:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
3,85171458,303,3025408,2006-01-02,2006-01-02 03:52:00+00:00,03:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
4,85171512,303,3025408,2006-01-02,2006-01-02 04:52:00+00:00,04:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
5,85171554,303,3025408,2006-01-02,2006-01-02 05:52:00+00:00,05:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
6,85171619,303,3025408,2006-01-02,2006-01-02 06:52:00+00:00,06:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
7,85171661,303,3025408,2006-01-02,2006-01-02 07:52:00+00:00,07:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
8,85171702,303,3025408,2006-01-02,2006-01-02 08:52:00+00:00,08:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN
9,85171741,303,3025408,2006-01-02,2006-01-02 09:52:00+00:00,09:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,NaN,NaT,NaN


In [95]:
##store
min_fi02 = rslt_df_3d_fi02_idata['value_source_value'].min()
start_unst_datetime  = ""
uns_len = 0
#print (min_fi02)

# loop through the rows using iterrows()
for index, row in rslt_df_3d_fi02_idata.iterrows():
  current_fi02 = rslt_df_3d_fi02_idata.loc[index, 'value_source_value']
  #print("---------------")
  #print("current_fi02=" + str(current_fi02))
  delta_min_fi02 = (current_fi02 - min_fi02)
  #print("delta_min_fi02=" + str(delta_min_fi02))

  rslt_df_3d_fi02_idata.loc[index, 'delta_min_fi02'] = delta_min_fi02
  if (delta_min_fi02 >= 20):
    unst_datetime = rslt_df_3d_fi02_idata.loc[index, 'measurement_datetime']
    if (start_unst_datetime == ""):
      start_unst_datetime = unst_datetime
    delta_unst_dt = unst_datetime - start_unst_datetime
    delta_unst_days = delta_unst_dt.days

    rslt_df_3d_fi02_idata.loc[index, 'delta_unst_dt'] = delta_unst_dt
    rslt_df_3d_fi02_idata.loc[index, 'delta_unst_days'] = delta_unst_days

    #print ("prev_fi02 " + str(prev_fi02) + " (delta_min_fi02)=" + str(delta_min_fi02))
    #print("delta_unst_dt=" + str(delta_unst_dt) + "")
    #print("delta_unst_days=" + str(delta_unst_days) + "")

    if (delta_unst_days >= 2):
      print ("str(delta_unst_days)=" + str(delta_unst_days))
    uns_len += 1
    #rslt_df_3d_fi02_idata['uns_len'] = uns_len
  else:
    start_unst_datetime = ""
  prev_fi02 = current_fi02

rslt_df_3d_fi02_idata.head(100)


,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_time,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,...,unit_source_concept_id,value_source_value,measurement_event_id,meas_event_field_concept_id,date_min,date_max,diff,delta_min_fi02,delta_unst_dt,delta_unst_days
0,85171342,303,3025408,2006-01-02,2006-01-02 00:54:00+00:00,00:54:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,1,0 days 00:00:00,0
1,85171383,303,3025408,2006-01-02,2006-01-02 01:52:00+00:00,01:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,1,0 days 00:00:00,0
2,85171421,303,3025408,2006-01-02,2006-01-02 02:52:00+00:00,02:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,1,0 days 00:00:00,0
3,85171458,303,3025408,2006-01-02,2006-01-02 03:52:00+00:00,03:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,1,0 days 00:00:00,0
4,85171512,303,3025408,2006-01-02,2006-01-02 04:52:00+00:00,04:52:00,32817,NaN,40.0,NaN,...,8554.0,40,NaN,NaN,2005-12-31,2006-01-23,23,1,0 days 00:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,85176102,303,3025408,2006-01-06,2006-01-06 18:52:00+00:00,18:52:00,32817,NaN,65.0,NaN,...,8554.0,65,NaN,NaN,2005-12-31,2006-01-23,23,26,0 days 18:27:00,0
96,85176145,303,3025408,2006-01-06,2006-01-06 19:52:00+00:00,19:52:00,32817,NaN,66.0,NaN,...,8554.0,66,NaN,NaN,2005-12-31,2006-01-23,23,27,0 days 19:27:00,0
97,85176197,303,3025408,2006-01-06,2006-01-06 20:52:00+00:00,20:52:00,32817,NaN,65.0,NaN,...,8554.0,65,NaN,NaN,2005-12-31,2006-01-23,23,26,0 days 20:27:00,0
98,85176234,303,3025408,2006-01-06,2006-01-06 21:52:00+00:00,21:52:00,32817,NaN,65.0,NaN,...,8554.0,65,NaN,NaN,2005-12-31,2006-01-23,23,26,0 days 21:27:00,0
